In [ ]:
import os
import sys
import math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


sys.path.append('../..')
import model.Baseline as baseline
import model.my_model as mymodel
import model.util_loss as util_ls
import model.util_dataloader as util_dl
import model.util_model as util_md
import data_process.util_data as util_dt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")

# local
file_save_dir = r'C:\Users\29492\Desktop\exp12'
file_load_dir = r'D:\dataset\DataFrames\REDD'
# server
# file_save_dir = r'../../data'
# file_load_dir = r'../../data'

dm = util_md.DictManager(file_save_dir)
mwm = util_md.ModelWeightManager(file_save_dir)

In [ ]:
class Model_Deeper(nn.Module):

    def __init__(self, in_seq_dim, out_seq_dim=1) -> None:
        super(Model_Deeper, self).__init__()
        self.in_seq_length = in_seq_dim
        self.out_seq_dim = out_seq_dim

        self.cnn = nn.Sequential(
            nn.ReplicationPad1d((1,1)),
            nn.Conv1d(in_channels=1, out_channels=30, kernel_size=3, stride=1),
            nn.ReLU(True),
            nn.ReplicationPad1d((2,2)),
            nn.Conv1d(in_channels=30, out_channels=90, kernel_size=4, stride=2),
            nn.ReLU(True),
            nn.ReplicationPad1d((2,2)),
            nn.Conv1d(in_channels=90, out_channels=180, kernel_size=5, stride=3),
            nn.ReLU(True),
            nn.ReplicationPad1d((3,3)),
            nn.Conv1d(in_channels=180, out_channels=360, kernel_size=6, stride=3),
            nn.ReLU(True),
            nn.ReplicationPad1d((3,5)),
            nn.Conv1d(in_channels=360, out_channels=720, kernel_size=7, stride=3),
            nn.ReLU(True)
        )

        output_channel, output_length = util_md.compute_output_shape(self.cnn,[1,in_seq_dim])
        self.flatten_size = output_channel*output_length

        self.clas = nn.Sequential(
            nn.Linear(self.flatten_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, out_seq_dim)
        )

        self.reg = nn.Sequential(
            nn.Linear(self.flatten_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, out_seq_dim)
        )

        self.act = nn.Sigmoid()

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(-1,self.flatten_size)
        output_clas = self.clas(x)
        output_reg = self.reg(x) * self.act(output_clas)
        return output_clas, output_reg


class Model_complex(nn.Module):

    def __init__(self, in_seq_dim, out_seq_dim=1) -> None:
        super(Model_complex, self).__init__()
        self.in_seq_length = in_seq_dim
        self.out_seq_dim = out_seq_dim

        self.cnn = nn.Sequential(
            nn.ReplicationPad1d((1,1)),
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, stride=1),
            nn.ReLU(True),
            nn.ReplicationPad1d((2,2)),
            nn.Conv1d(in_channels=32, out_channels=128, kernel_size=5, stride=2),
            nn.ReLU(True),
            nn.ReplicationPad1d((3,3)),
            nn.Conv1d(in_channels=128, out_channels=524, kernel_size=7, stride=3),
            nn.ReLU(True)
        )

        self.gru1 = nn.GRU(input_size=524, hidden_size=1024, bidirectional=True, batch_first=True)
        self.dropout1 = nn.Dropout(0.5)

        output_channel, output_length = util_md.compute_output_shape(self.cnn,[1,in_seq_dim])
        self.flatten_size = 1024*2*output_length

        self.clas = nn.Sequential(
            nn.Linear(self.flatten_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, out_seq_dim)
        )

        self.reg = nn.Sequential(
            nn.Linear(self.flatten_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, out_seq_dim)
        )

        self.act = nn.Sigmoid()

    def forward(self, x):
        x = self.cnn(x)
        x,_ = self.gru1(x.transpose(-2,-1))
        x = self.dropout1(x)
        x = x.flatten(start_dim=1)
        output_clas = self.clas(x)
        output_reg = self.reg(x) * self.act(output_clas)
        return output_clas, output_reg



class Model_Combined(nn.Module):

    def __init__(self, in_seq_dim, out_seq_dim=1) -> None:
        super(Model_Combined, self).__init__()
        self.in_seq_length = in_seq_dim
        self.out_seq_dim = out_seq_dim

        self.cnn = nn.Sequential(
            nn.ReplicationPad1d((1,1)),
            nn.Conv1d(in_channels=1, out_channels=30, kernel_size=3, stride=1),
            nn.ReLU(True),
            nn.ReplicationPad1d((2,2)),
            nn.Conv1d(in_channels=30, out_channels=90, kernel_size=4, stride=2),
            nn.ReLU(True),
            nn.ReplicationPad1d((2,2)),
            nn.Conv1d(in_channels=90, out_channels=180, kernel_size=5, stride=3),
            nn.ReLU(True),
            nn.ReplicationPad1d((3,3)),
            nn.Conv1d(in_channels=180, out_channels=360, kernel_size=6, stride=3),
            nn.ReLU(True),
            nn.ReplicationPad1d((3,5)),
            nn.Conv1d(in_channels=360, out_channels=720, kernel_size=7, stride=3),
            nn.ReLU(True)
        )

        self.gru1 = nn.GRU(input_size=720, hidden_size=720*3, bidirectional=True, batch_first=True)
        self.dropout1 = nn.Dropout(0.5)

        output_channel, output_length = util_md.compute_output_shape(self.cnn,[1,in_seq_dim])
        self.flatten_size = 720*3*2*output_length

        self.clas = nn.Sequential(
            nn.Linear(self.flatten_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, out_seq_dim)
        )

        self.reg = nn.Sequential(
            nn.Linear(self.flatten_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, out_seq_dim)
        )

        self.act = nn.Sigmoid()

    def forward(self, x):
        x = self.cnn(x)
        x,_ = self.gru1(x.transpose(-2,-1))
        x = self.dropout1(x)
        x = x.flatten(start_dim=1)
        output_clas = self.clas(x)
        output_reg = self.reg(x) * self.act(output_clas)
        return output_clas, output_reg

In [ ]:
model = Model_Combined(599,14)
mwm.save_model_weight(model,'test')

In [ ]:
dataframe_path = os.path.join(file_load_dir, r'house_1.csv')
df = pd.read_csv(dataframe_path)
df.set_index('time', inplace=True)
# df.describe(percentiles=[0.10,0.25,0.50,0.75,0.8,0.85,0.9,0.95])


house_1_cols = ['oven-3', 'oven-4', 'refrigerator-5', 'dishwaser-6', 'lighting-9', 'washer_dryer-10', 'microwave-11',
                'bathroom_gfi-12', 'electric_heat-13', 'stove-14', 'lighting-17', 'lighting-18', 'washer_dryer-19', 'washer_dryer-20']
filtered_columns = df[house_1_cols]
# filtered_columns.describe(percentiles=[0.10,0.25,0.50,0.6,0.75,0.8,0.85,0.9,0.95])


apps_name = filtered_columns.columns.to_list()
print(apps_name)


filtered_columns['sum'] = filtered_columns.sum(axis=1)
norm_data, min_val, max_val = util_dt.normalization_interval(filtered_columns.to_numpy(), 0, 1)
main = norm_data[:,-1]
apps = norm_data[:,:-1]
print(f'min: {min_val}; max: {max_val}')
print(np.min(main), np.max(main))
print(np.min(apps), np.max(apps))
util_dt.print_shape(main, apps)


on_threshold = [50, 50, 50, 50, 20, 50, 50, 50, 50, 50, 20, 20, 50, 50]
sliding_window_len = 599

In [ ]:
in_seq_l = sliding_window_len
out_dim = len(apps_name)
print(f'in_seq_l: {in_seq_l}; out_dim: {out_dim}')

loss_factor=max_val-min_val
print(f'loss_factor:{loss_factor}')


metric_dict_one2one = {'MAE(apps)':util_ls.MAE(single_class=True),
                'SAE':util_ls.SignalAggregateError(single_class=True, period_len=450),
                'MAE(offon)':util_ls.MAE_off_on(single_class=True)}
metric_dict_one2all = {'MAE(apps)':util_ls.MAE(single_class=False),
                       'MAE(mean)':util_ls.MAE(single_class=True),
                'SAE(apps)':util_ls.SignalAggregateError(single_class=False, period_len=450),
                'SAE(mean)':util_ls.SignalAggregateError(single_class=True, period_len=450),
                'MAE(offon)':util_ls.MAE_off_on(single_class=False)}


apps_to_train = ['refrigerator-5', 'dishwaser-6', 'washer_dryer-10', 'microwave-11', 'bathroom_gfi-12']
for app in apps_to_train:
    print(apps_name.index(app), app)

In [ ]:
w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=apps, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
w_main = np.expand_dims(w_main, axis=1)
w_apps_sum = w_apps.sum(axis=-1)
w_apps_sum = np.expand_dims(w_apps_sum, axis=1)
w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
util_dt.print_shape(w_main, w_apps, w_apps_sum, w_apps_on)

In [ ]:
epoch_n = 50
b_s = 450*20
eval_b_s = 450*20

exp_num = 2

dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                            torch.from_numpy(w_apps_on).to(torch.float32),
                            torch.from_numpy(w_apps_sum).to(torch.float32),
                            torch.from_numpy(w_apps).to(torch.float32))
dataset.describe()
subset_1, subset_2 = util_dl.split_Dataset_orderly(dataset, [0.8,0.2])
trainset, valset = util_dl.split_to_DataLoader_randomly(dataset=subset_1, split_ratio=[0.75,0.25], batch_size=b_s, shuffle_flag=True)
testset = util_dl.just_to_DataLoader(dataset=subset_2, batch_size=eval_b_s, shuffle_flag=False)

for i in range(exp_num):

    model = mymodel.Model_final(in_seq_l, out_dim)

    score_train_vali, _ = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, epoch_n, trainset, valset,
                                        loss_reg_factor=loss_factor, return_metric_reg_dict=metric_dict_one2all, decay_params=[10,0.5])

    score_test, _  = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, valset=testset, loss_reg_factor=loss_factor,
                                        return_metric_reg_dict=metric_dict_one2all)

    total_record = {'train_vali':score_train_vali, 'test':score_test}
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='REDD',model=type(model).__name__,appname='all',date=f'1228-{i}')
    dm.save_dict(total_record, filename)
    mwm.save_model_weight(model, filename)

del trainset, valset, testset